In [4]:
pip -q install lxml

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip -q install geopy

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip -q install folium

Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import lxml
import numpy as np
import requests # library to handle requests
import folium # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [8]:
gs_data=pd.read_csv('https://cocl.us/Geospatial_data')
df_wiki=pd.read_html('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050')[0]

In [9]:
df_wiki=df_wiki[df_wiki['Borough'].str.find("Not assigned")==-1]
df_wiki.reset_index(inplace=True, drop=True)
df_wiki.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)
df_wiki['Latitude']=pd.Series(dtype=float)
df_wiki['Longitude']=pd.Series(dtype=float)
df_wiki.head(1)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,NaN,NaN


##### Build table with neighborhoods and Lat/Long by Postcode

In [10]:
df_wiki_gs=df_wiki.copy()
for i in range(0,df_wiki_gs.shape[0]):  
    if df_wiki_gs.iloc[i,2].find('Not assigned')!=-1:
        df_wiki_gs.iloc[i,2]=df_wiki.iloc[i,1]
    for j in range(0, gs_data.shape[0]):
        if df_wiki_gs.iloc[i,0]==gs_data.iloc[j,0]:
            df_wiki_gs.iloc[i,3]=gs_data.iloc[j,1]
            df_wiki_gs.iloc[i,4]=gs_data.iloc[j,2]
    k=i
    while i<df_wiki_gs.shape[0]-1 and df_wiki_gs.iloc[i,0]==df_wiki_gs.iloc[i+1,0]:
        df_wiki_gs.iloc[k,2]=df_wiki_gs.iloc[k,2]+', '+df_wiki_gs.iloc[i+1,2]
        df_wiki_gs.iloc[i+1,2]='Delete'
        i=i+1        

In [46]:
df_wiki_gs=df_wiki_gs[~df_wiki_gs['Neighborhood'].str.contains('Delete', regex=False)]
df_wiki_gs.reset_index(inplace=True, drop=True)
df_wiki_gs.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


##### We will be using the original dataframe for clustering, because the dataframe we obtained above doesn't identify latitude and longitude by neighborhood, but only by Postcode

In [12]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_wiki['Borough'].unique()),
        df_wiki_gs.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [28]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'The geograpical coordinate of {address} are latitude {latitude}, longitude {longitude}')

The geograpical coordinate of Toronto, Canada are latitude 43.653963, longitude -79.387207


In [29]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, postcode in zip(df_wiki_gs['Latitude'], df_wiki_gs['Longitude'], df_wiki_gs['Borough'], df_wiki_gs['Postcode']):
    label = '{}, {}'.format(postcode, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [30]:
CLIENT_ID =  'ZFJACRK522OEELKT1D2KXZV4CRMLOZJDUVWVUHYQ0GNMD11R' # your Foursquare ID
CLIENT_SECRET =  'ETDNQOIP3E0WLCC0HPAEUZ5E4B3YIAHIB2XTMSCLZLTN0ZOI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZFJACRK522OEELKT1D2KXZV4CRMLOZJDUVWVUHYQ0GNMD11R
CLIENT_SECRET:ETDNQOIP3E0WLCC0HPAEUZ5E4B3YIAHIB2XTMSCLZLTN0ZOI


In [47]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):   
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request

        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Postcode Latitude', 
                  'Postcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [48]:
toronto_venues = getNearbyVenues(names=df_wiki_gs['Postcode'],latitudes=df_wiki_gs['Latitude'],longitudes=df_wiki_gs['Longitude'])

In [49]:
print(toronto_venues.shape)

(2247, 7)


In [34]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Postcode'] = toronto_venues['Postcode'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Postcode,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
toronto_grouped = toronto_onehot.groupby('Postcode').mean().reset_index()
toronto_grouped.head(1)

,Postcode,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postcode'] = toronto_grouped['Postcode']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Fast Food Restaurant,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Falafel Restaurant
1,M1C,Bar,History Museum,Home Service,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
2,M1E,Spa,Mexican Restaurant,Electronics Store,Breakfast Spot,Rental Car Location,Medical Center,Bank,Intersection,Distribution Center,Dog Run
3,M1G,Coffee Shop,Pharmacy,Korean Restaurant,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
4,M1H,Bank,Hakka Restaurant,Fried Chicken Joint,Lounge,Thai Restaurant,Athletics & Sports,Caribbean Restaurant,Bakery,Gas Station,Dog Run


In [38]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Postcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [39]:
# add clustering labels

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_.astype(int))

In [52]:
toronto_merged = df_wiki_gs

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Postcode'), on='Postcode')
print(toronto_merged[toronto_merged['Cluster Labels'].isna()])
toronto_merged=toronto_merged.dropna(axis=0)
toronto_merged.head() # check the last columns!

   Postcode      Borough                                       Neighborhood  \
5       M9A    Etobicoke                                   Islington Avenue   
11      M9B    Etobicoke  Cloverdale, Islington, Martin Grove, Princess ...   
52      M2M   North York                            Newtonbrook, Willowdale   
95      M1X  Scarborough                                        Upper Rouge   

     Latitude  Longitude  Cluster Labels 1st Most Common Venue  \
5   43.667856 -79.532242             NaN                   NaN   
11  43.650943 -79.554724             NaN                   NaN   
52  43.789053 -79.408493             NaN                   NaN   
95  43.836125 -79.205636             NaN                   NaN   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
5                    NaN                   NaN                   NaN   
11                   NaN                   NaN                   NaN   
52                   NaN                   NaN           

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,4.0,Park,Food & Drink Shop,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Intersection,Coffee Shop,Portuguese Restaurant,Hockey Arena,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,1.0,Coffee Shop,Park,Pub,Bakery,Mexican Restaurant,Café,Theater,Yoga Studio,Shoe Store,Hotel
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,1.0,Clothing Store,Furniture / Home Store,Accessories Store,Coffee Shop,Shoe Store,Miscellaneous Shop,Boutique,Event Space,Gift Shop,Vietnamese Restaurant
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,1.0,Coffee Shop,Park,Chinese Restaurant,Sushi Restaurant,Bank,Bar,Portuguese Restaurant,Café,Diner,Beer Bar


In [41]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]

colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
     
map_clusters

In [53]:
toronto_merged['Cluster Labels'].tail()

98     4.0
99     1.0
100    1.0
101    1.0
102    1.0
Name: Cluster Labels, dtype: float64

In [43]:
toronto_merged['Cluster Labels'].unique()

array([4., 1., 0., 3., 2.])

In [54]:
toronto_merged.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,4.0,Park,Food & Drink Shop,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Intersection,Coffee Shop,Portuguese Restaurant,Hockey Arena,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,1.0,Coffee Shop,Park,Pub,Bakery,Mexican Restaurant,Café,Theater,Yoga Studio,Shoe Store,Hotel
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,1.0,Clothing Store,Furniture / Home Store,Accessories Store,Coffee Shop,Shoe Store,Miscellaneous Shop,Boutique,Event Space,Gift Shop,Vietnamese Restaurant
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,1.0,Coffee Shop,Park,Chinese Restaurant,Sushi Restaurant,Bank,Bar,Portuguese Restaurant,Café,Diner,Beer Bar


What are the 5 most common venues in the 5 clusters ?

In [45]:
#pd.set_option('expand_frame_repr', True)
for i in range(0,5):
   print('\033[1;43m'+'Cluster',i,'\033[0m')
   for j in (3,6):
        print (', '.join(toronto_merged.loc[toronto_merged['Cluster Labels'] == i, toronto_merged.columns[[0,1] + list(range(5,10))][j]].head(5)))

Cluster 0 
Fast Food Restaurant, Rental Car Location
Discount Store, Doner Restaurant
Cluster 1 
Intersection, Coffee Shop, Clothing Store, Coffee Shop, Gym / Fitness Center
Hockey Arena, Bakery, Coffee Shop, Sushi Restaurant, Japanese Restaurant
Cluster 2 
Garden
Diner
Cluster 3 
Cafeteria
Diner
Cluster 4 
Park, Park, Convenience Store, Airport, Bakery
Donut Shop, Donut Shop, Drugstore, Drugstore, Yoga Studio
